In [1]:
from openml import tasks

import torch

import sklearn
from sklearn.model_selection import train_test_split, StratifiedKFold

from classes import Dataset, GroupStructure, WeightClipper
from functions import run_HPO_CV

In [2]:
oml_task_diabetes = tasks.get_task(37)

In [3]:
X, y, categorical_indicator, attribute_names = oml_task_diabetes.get_dataset().get_data()

In [4]:
X

,preg,plas,pres,skin,insu,mass,pedi,age,class
0,6.0,148.0,72.0,35.0,0.0,33.6,0.627,50.0,tested_positive
1,1.0,85.0,66.0,29.0,0.0,26.6,0.351,31.0,tested_negative
2,8.0,183.0,64.0,0.0,0.0,23.3,0.672,32.0,tested_positive
3,1.0,89.0,66.0,23.0,94.0,28.1,0.167,21.0,tested_negative
4,0.0,137.0,40.0,35.0,168.0,43.1,2.288,33.0,tested_positive
...,...,...,...,...,...,...,...,...,...
763,10.0,101.0,76.0,48.0,180.0,32.9,0.171,63.0,tested_negative
764,2.0,122.0,70.0,27.0,0.0,36.8,0.340,27.0,tested_negative
765,5.0,121.0,72.0,23.0,112.0,26.2,0.245,30.0,tested_negative
766,1.0,126.0,60.0,0.0,0.0,30.1,0.349,47.0,tested_positive


In [5]:
Xy = X.copy()

In [6]:
gs = GroupStructure(
    {0, 1, 2, 3, 4, 5, 6, 7},
    {0, 1},
    ((2, 5), 1),
    ((4,), 0),
    ((7, 3, 6), 1)
)

tmp = Dataset(
    X=Xy.loc[:, Xy.columns != 'class'],
    y=Xy.loc[:, 'class'],
    class_pos='tested_positive',
    group_structure=gs
)
len(tmp)
tmp[2]

gs.get_unconstrained_features()

{4}

In [7]:
categorical_indicator

[False, False, False, False, False, False, False, False, True]

In [8]:
# outer
data_train_test, data_val = train_test_split(
    Xy,
    train_size=2/3,
    shuffle=True,
    stratify=Xy.loc[:, 'class']
)

# reset indices as StratifiedKFold assumes 0-(n-1) index
data_train_test = data_train_test.reset_index(drop=True)
data_val = data_val.reset_index(drop=True)

# inner
cv_inner = StratifiedKFold(
    n_splits=5,
    shuffle=False  # TODO: set to True
)

'''
in each inner fold, run EAGGA + find an optimal pareto front of configurations for this specific fold
then in the outer fold (holdout), compare each fold-specific optimal pareto front + select optimum
'''
monotonicity_weight_clipper = WeightClipper(0, None)  # enforce monotonicity by clipping weights to [0, infty) after each epoch (in def train)
run_HPO_CV(cv_inner, data_train_test, epochs=10, batch_size=8, weight_clipper=monotonicity_weight_clipper)

running HPO: 3 total_layers, 3 nodes per hidden layer
fold 1 / 5
epoch 0 / 10, eval loss 2.3989868611097336
epoch 1 / 10, train loss 1.5397669478104665
epoch 1 / 10, eval loss 1.3948207600758626
epoch 2 / 10, train loss 0.9595855548977852
epoch 2 / 10, eval loss 0.860941258760599
epoch 3 / 10, train loss 0.7439251106518966
epoch 3 / 10, eval loss 0.7051993104127737
epoch 4 / 10, train loss 0.6811506496025965
epoch 4 / 10, eval loss 0.6609895779536321
epoch 5 / 10, train loss 0.6630716811005886
epoch 5 / 10, eval loss 0.649787024809764
epoch 6 / 10, train loss 0.659400252195505
epoch 6 / 10, eval loss 0.6486282967604123
epoch 7 / 10, train loss 0.658531744319659
epoch 7 / 10, eval loss 0.6477387020221124
epoch 8 / 10, train loss 0.6570493487211374
epoch 8 / 10, eval loss 0.6468521769230182
epoch 9 / 10, train loss 0.656177349961721
epoch 9 / 10, eval loss 0.6463244832479037
epoch 10 / 10, train loss 0.655495072213503
epoch 10 / 10, eval loss 0.6458642138884618
(0.5538971807628524, 0.75,